# Rapport E1, Étude

## Fonctions et import

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport


## Importation des données
Pour étudier le contenu du dataset, on va supprimer la colonne id et rajouter une colonne IMC pour traquer les valeur abérantes.

In [8]:
df= pd.read_csv("./datas/cardio_train.csv",sep= ";")
#df.columns
# suppression de la colonne id
df.drop("id", axis= 1, inplace= True)

df["IMC"]= df.weight/((df.height/100)**2)
# suppression de valeur abbérante IMC
#df= df[df.IMC > 14]
#df= df[df.IMC < 50]

# suppression valeurs abbérantes pression artérielle
#df= df[df.ap_hi > 50]
#df= df[df.ap_hi < 300]

#df= df[df.ap_lo > 20]
#df= df[df.ap_lo < 300]

# suppression de la taille et du poids redondante avec l'IMV
#df.drop("height", axis= 1, inplace= True)
#df.drop("weight", axis= 1, inplace= True)

df

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,IMC
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,21.967120
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,34.927679
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,23.507805
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,28.710479
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,23.011177
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,2,168,76.0,120,80,1,1,1,0,1,0,26.927438
69996,22601,1,158,126.0,140,90,2,2,0,0,1,1,50.472681
69997,19066,2,183,105.0,180,90,3,1,0,1,0,1,31.353579
69998,22431,1,163,72.0,135,80,1,2,0,0,0,1,27.099251


In [9]:
df.height.max(), df.height.min(), df.weight.max(), df.weight.min(), df.ap_hi.max(), df.ap_hi.min(), df.ap_lo.max(), df.ap_lo.min()

(250, 55, 200.0, 10.0, 16020, -150, 11000, -70)

In [10]:
analyse= ProfileReport(df)

In [11]:
analyse.to_file(output_file="cardio.html")

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
cte_age_an_jour= 365,242199
